# Python vs Matlab calibration comparison

## Objective: compare calibration checkerboard detection in python (opencv) & matlab (toolbox)

### Test 1: single image, intrinsic camera calibration

In [ ]:
import numpy as np
import cv2
from cv2 import aruco
import glob
import time
import pandas as pd

from scipy.io import loadmat, savemat
from src.calibration.new.utils import get_chessboard_coordinates, imshow
        
# extra imports
import os
import matplotlib.pyplot as plt

In [ ]:
# interactive plot widgit:
%matplotlib widget

In [ ]:
# test parameters

rows = 6
cols = 9
square_size_mm = 23

width = 1920
height = 1200

# note: in this case frame_id is a string with 4 digits, leading zeros if necessary
# frame_id: str = '0034'
# camera_name: str = 'Camera 4'
# 1: cam1, f34
# 2: cam4, f17
frames = [
    '0001','0002', '0006','0007', '0008','0009', '0010', '0011', '0012', '0013'
]

f = lambda frame: f'/Users/caxon/olveczky/dannce_data/setupCal11_010324/Intrinsics/Camera 4/Basler_a2A1920-160ucPRO__40125727__20240103_153235585_{frame}.tiff'

image_paths = [f(fr) for fr in frames]

# disable for mlutiple images
show_img = False

objpoints = get_chessboard_coordinates(chessboard_rows=rows,
           chessboard_cols=cols, square_size_mm=square_size_mm)

all_raw_images = []
# load raw image(s)
for image_path in image_paths:
    raw_image = np.zeros((height, width, 3), dtype=np.uint8)
    raw_image = cv2.imread(image_path)
    all_raw_images.append(raw_image)

print("ALL IMAGES LOADED", len(all_raw_images))

all_imgpoints = []
all_objpoints = []

for i in range(len(all_raw_images)):
    # find points
    this_img = all_raw_images[i][:,:,:].copy()
    gray = cv2.cvtColor(this_img, cv2.COLOR_BGR2GRAY)
    success, corner_coords = cv2.findChessboardCorners(gray, (cols,rows), None)
    
    if success == True:     
        print(f"#{i}[{frames[i]}]: SUCCESS")
        imgpoints = corner_coords
        all_imgpoints.append(imgpoints)
        all_objpoints.append(objpoints)
        if show_img is True:
            cv2.drawChessboardCorners(this_img, (cols,rows), corner_coords, True)
            imshow(this_img)
    else:
        print(f"#{i}[{frames[i]}]: FAILURE")

imgpoints = imgpoints.squeeze()

# check the shapes of resulting points
print("OBJ POINTS SHAPE:", objpoints.shape)
print("IMG POINTS SHAPE:", imgpoints.shape)

In [ ]:
rpe, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(all_objpoints, all_imgpoints, gray.shape[::-1] , None, None, flags=cv2.CALIB_FIX_K3)

np.set_printoptions(suppress=True, precision=4)

print(f"error: {rpe}\n")
print(f"mtx:\n{mtx}\n")
print(f"dist (k1, k2, p1, p2, k3?):\n{dist.squeeze()}\n")
print(f"rvecs[0]: {rvecs[0].squeeze()}\n")
print(f"tvecs[0]: {tvecs[0].squeeze()}\n")

rmats = []
for rvec in rvecs:
    rmat, _jac = cv2.Rodrigues(rvec)
    rmats.append(rmat)
print(f"revec rot. matrix[0]:\n{rmats[0]}\n")

In [ ]:
# load calibration session points from matlab:

matfile = loadmat("/Users/caxon/olveczky/newsdannce/calibFromMatlabCam4.mat")
mat_mtx = matfile.get("intrinsicMatrix")
mat_dist = matfile.get("distortionCoefficients")
mat_tvecs = [x[0].reshape((3,1)) for x in (matfile.get("t_vecs"))]
mat_rmats = [x[0] for x in matfile.get("r_mats")]

In [ ]:
mat_rmats

In [ ]:
# python values
x1 = [p[0][0] for p in tvecs]
y1 = [p[1][0] for p in tvecs]
z1 = [p[2][0] for p in tvecs]

# matlab values
x2 = [p[0][0] for p in mat_tvecs]
y2 = [p[1][0] for p in mat_tvecs]
x2 = [p[2][0] for p in mat_tvecs]

ax = plt.figure().add_subplot(projection='3d')
ax.scatter(xs=x1, ys=y1, zs=z1, c='red')